# Matrix Factorization

In [2]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.01, Xinit = None, Winit = None, 
                 learning_rate = 0.01, max_iter = 1000, print_every = 100, user_based = 1, NN = 0):
        self.Y_raw_data = Y_data
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.user_based = user_based
        self.NN = NN 
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        
        if Xinit is None: 
            if self.NN:
                self.X = np.random.rand(self.n_items, K)
            else:
                self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit 
        
        if Winit is None: 
            if self.NN:
                self.W = np.random.rand(K, self.n_users)
            else:
                self.W = np.random.randn(K, self.n_users)
        else: 
            self.W = Winit
            
        #self.all_users = self.Y_data[:,0] # all users (may be duplicated)
        self.n_ratings = Y_data.shape[0]
        # normalized data
        self.Y_data_n = self.Y_raw_data.copy()

    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))
        for n in xrange(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
            
    
    def loss(self):
        L = 0 
        for i in xrange(self.Y_data_n.shape[0]):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
            
        # regularization, don't ever forget this 
        L /= self.n_ratings
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L 

    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user n, and the corresponding ratings
        """
        # y = self.Y_data_n[:,0] # all users (may be duplicated)
        # item indices rated by user_id
        # we need to +1 to user_id since in the rate_matrix, id starts from 1 
        # while index in python starts from 0
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # index starts from 0 
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item m and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
    
    
    def updateX(self):
        for m in xrange(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
        self.X = np.maximum(0, self.X)
    
    
    def updateW(self):
        for n in xrange(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            grad_wn = -Xn.T.dot(ratings- Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
        self.W = np.maximum(0, self.W)
    
    
    def fit(self):
#         self.normalize_Y()
        self.Y_data_n = self.Y_raw_data
        for it in xrange(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                print 'iter =', it + 1, ', loss =', self.loss()
    
    
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        return self.X[i, :].dot(self.W[:, u])
    
        if self.user_based:
            return self.X[i, :].dot(self.W[:, u]) + self.mu[u]
        return self.X[i, :].dot(self.W[:, u]) + self.mu[i]
        
    
    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in xrange(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
        
r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
Y_data = ratings.as_matrix()


rs = MF(Y_data, K = 2, max_iter = 10000, print_every = 1000, NN = 1)

rs.fit()
rs.pred(6, 1)

iter = 1000 , loss = 0.316681549148
iter = 2000 , loss = 0.13563610798
iter = 3000 , loss = 0.098945941908
iter = 4000 , loss = 0.0869023843657
iter = 5000 , loss = 0.0822775131126
iter = 6000 , loss = 0.0807994848759
iter = 7000 , loss = 0.0802245607348
iter = 8000 , loss = 0.0799599023531
iter = 9000 , loss = 0.0798188919094
iter = 10000 , loss = 0.0797320256677


1.2282500452670797

In [7]:
print rs.X.dot(rs.W)
print rs.Y_data_n

[[  2.41234576e-06   1.22989011e-06   9.97186671e-07 ...,   1.48579090e-06
    1.45113626e-06   1.96613214e-06]
 [  1.50790372e-06   7.96783139e-07   4.95255795e-07 ...,   9.24645494e-07
    8.63795236e-07   1.17765914e-06]
 [  1.03992612e-06   5.87427985e-07   3.51906682e-07 ...,   5.74799363e-07
    6.27326546e-07   7.75386864e-07]
 ..., 
 [  8.40426204e-07   4.49956981e-07   3.06062187e-07 ...,   4.79609329e-07
    4.52604384e-07   6.96525451e-07]
 [  7.36206050e-07   4.00145408e-07   2.66860658e-07 ...,   4.84409569e-07
    5.01604590e-07   6.47818003e-07]
 [  1.05994714e-06   4.94480203e-07   3.94747937e-07 ...,   6.79916872e-07
    6.33373150e-07   9.03335090e-07]]
[[        0         0         5 874965758]
 [        0         1         3 876893171]
 [        0         2         4 878542960]
 ..., 
 [      942      1187         3 888640250]
 [      942      1227         3 888640275]
 [      942      1329         3 888692465]]


# Áp dụng lên MovieLens 100k

In [4]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.as_matrix()
rate_test = ratings_test.as_matrix()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [5]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, 
        max_iter = 100, user_based = 1,NN = 1)
rs.fit()
# evaluate on test data
n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in xrange(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print 'Item-item CF, RMSE =', RMSE

iter = 10 , loss = 8.05259737799
iter = 20 , loss = 7.80140862387
iter = 30 , loss = 7.39417946825
iter = 40 , loss = 7.12528097703
iter = 50 , loss = 6.98049452199
iter = 60 , loss = 6.90820896212
iter = 70 , loss = 6.87326760762
iter = 80 , loss = 6.85661964607
iter = 90 , loss = 6.84873730732
iter = 100 , loss = 6.84501414989
Item-item CF, RMSE = 3.76118952705


In [19]:
# rs = MF(rate_train, K = 2, lam = 0, print_every = 10, learning_rate = 0.95, 
#         max_iter = 100, user_based = 0, Xinit = rs.X, Winit = rs.W)
rs = MF(rate_train, K = 500, lam = 0.1, print_every = 10, learning_rate = 1, 
        max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in xrange(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print 'Item-item CF, RMSE =', RMSE

iter = 10 , loss = 31.3256353273
iter = 20 , loss = 9.99306647393
iter = 30 , loss = 3.6224860534
iter = 40 , loss = 1.41663815287
iter = 50 , loss = 0.647698286083
iter = 60 , loss = 0.379580945089
iter = 70 , loss = 0.286092692681
iter = 80 , loss = 0.253495008616
iter = 90 , loss = 0.242128803778
iter = 100 , loss = 0.238165621765
Item-item CF, RMSE = 1.04980475284


# above


In [35]:
rs = MF(rate_train, K = 100, lam = 0.01, print_every = 50, learning_rate = 1, max_iter = 2, Xinit = rs.X, 
       Winit = rs.W)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in xrange(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print 'Item-item CF, RMSE =', RMSE

iter = 0 , loss = 0.25313267107
Item-item CF, RMSE = 1.06037990574


In [12]:
print rs.pred(1, 1)

6.68991073761


In [102]:
rs2 = MF(rate_train, K = 10, lam = 0.01, print_every = 20, learning_rate = 0.001, max_iter = 200)
rs2.fit()

iter = 0 , loss = 834968.255442
iter = 20 , loss = 63485.6626282
iter = 40 , loss = 42930.3071275
iter = 60 , loss = 37754.5380156
iter = 80 , loss = 35325.5213646
iter = 100 , loss = 33808.7359909
iter = 120 , loss = 32706.0820541
iter = 140 , loss = 31837.7213261
iter = 160 , loss = 31122.6640069
iter = 180 , loss = 30516.7608351


In [118]:
rs3 = MF(rate_train, K = 20, lam = 0.01, print_every = 20, \
         learning_rate = 0.1, max_iter = 500)
rs3.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in xrange(n_tests):
    pred = rs3.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print 'Item-item CF, RMSE =', RMSE

iter = 0 , loss = 18.2010515741
iter = 20 , loss = 17.2590357114
iter = 40 , loss = 16.4078277213
iter = 60 , loss = 15.6376045135
iter = 80 , loss = 14.9397357522
iter = 100 , loss = 14.3066209185
iter = 120 , loss = 13.7315513198
iter = 140 , loss = 13.2085928015
iter = 160 , loss = 12.7324857114
iter = 180 , loss = 12.2985592988
iter = 200 , loss = 11.9026582371
iter = 220 , loss = 11.5410793644
iter = 240 , loss = 11.2105170678
iter = 260 , loss = 10.9080160016
iter = 280 , loss = 10.6309300501
iter = 300 , loss = 10.3768866208
iter = 320 , loss = 10.1437555037
iter = 340 , loss = 9.92962165041
iter = 360 , loss = 9.73276132892
iter = 380 , loss = 9.55162119195
iter = 400 , loss = 9.38479986624
iter = 420 , loss = 9.23103172866
iter = 440 , loss = 9.08917258323
iter = 460 , loss = 8.95818699414
iter = 480 , loss = 8.83713706458
Item-item CF, RMSE = 4.05810732794


In [129]:
rs4 = MF(rate_train, K = 2, lam = 0.01, print_every = 20, \
         learning_rate = 0.5, max_iter = 500)
rs4.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in xrange(n_tests):
    pred = rs4.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print 'Item-item CF, RMSE =', RMSE

AttributeError: 'MF' object has no attribute 'Y_data'

\\[
\mathbf{w}_n = \mathbf{w}_n - \eta \left(-\frac{1}{s}\hat{\mathbf{X}}_n^T (\hat{\mathbf{y}}_n - \hat{\mathbf{X}}_n\mathbf{w}_n) + \lambda \mathbf{w}_n\right) ~~~~~(7)
\\]